In [27]:
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()
import numpy as np
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold,KFold,StratifiedGroupKFold,GroupKFold,train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD,PCA
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import os
import pickle
from rdkit import Chem
from rdkit.Chem import MACCSkeys, Descriptors
from rdkit.Chem.rdMolDescriptors import CalcNumRotatableBonds
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
from rdkit import DataStructs
from rdkit import RDLogger  
RDLogger.DisableLog('rdApp.*')  
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.optim import Adam

import warnings
warnings.filterwarnings('ignore')

In [28]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

### Getting features from rdkit

In [29]:
def compute_rdkit_feats(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    
    features = {}
    
    maccs = MACCSkeys.GenMACCSKeys(mol)
    
    for i in range(1, maccs.GetNumBits()):  # bit 0 is always 1
        features[f'MACCS_{i}'] = int(maccs.GetBit(i))

    # Descriptors
    descs = {
        'MolWt': Descriptors.MolWt(mol),
        'TPSA': Descriptors.TPSA(mol),
        'NumValenceElectrons': Descriptors.NumValenceElectrons(mol),
        'NumHeavyAtoms': Descriptors.HeavyAtomCount(mol),
        'NumRings': Descriptors.RingCount(mol),
        'NumRotatableBonds': CalcNumRotatableBonds(mol),
        'MolLogP': Descriptors.MolLogP(mol),
        'MolMR': Descriptors.MolMR(mol),
        'NumHAcceptors': Descriptors.NumHAcceptors(mol),
        'NumHDonors': Descriptors.NumHDonors(mol)
    }
    features.update(descs)

    return features

In [30]:
maccs_data = []

for smile in tqdm(train['SMILES']):
    feats = compute_rdkit_feats(smile)
    if feats is None:
        maccs_data.append(None)
    else:
        maccs_data.append(feats)
        
maccs_df = pd.DataFrame(maccs_data)
train_rdkit = pd.concat([train, maccs_df], axis=1)

  0%|          | 0/7973 [00:00<?, ?it/s]

In [31]:
train_rdkit.head()

,id,SMILES,Tg,FFV,Tc,Density,Rg,MACCS_1,MACCS_2,MACCS_3,...,MolWt,TPSA,NumValenceElectrons,NumHeavyAtoms,NumRings,NumRotatableBonds,MolLogP,MolMR,NumHAcceptors,NumHDonors
0,87817,*CC(*)c1ccccc1C(=O)OCCCCCC,NaN,0.374645,0.205667,NaN,NaN,0,0,0,...,232.323,26.30,92,17,1,8,3.98170,68.4145,2,0
1,106919,*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5...,NaN,0.370410,NaN,NaN,NaN,0,0,0,...,598.919,24.06,236,45,5,16,12.35960,193.3954,2,2
2,388772,*Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(...,NaN,0.378860,NaN,NaN,NaN,0,0,0,...,1003.207,122.27,364,73,10,15,14.21700,281.6006,9,0
3,519416,*Nc1ccc(-c2c(-c3ccc(C)cc3)c(-c3ccc(C)cc3)c(N*)...,NaN,0.387324,NaN,NaN,NaN,0,0,0,...,542.726,24.06,204,42,6,7,11.00768,179.8154,2,2
4,539187,*Oc1ccc(OC(=O)c2cc(OCCCCCCCCCOCC3CCCN3c3ccc([N...,NaN,0.355470,NaN,NaN,NaN,0,0,0,...,965.154,182.28,376,70,6,34,11.84500,266.9958,14,0


### Dataset Class

In [17]:
class MultiOPDataset(Dataset):
    def __init__(self, df, feat_cols, targ_cols):
        self.X = df[feat_cols].values.astype('float32')
        self.y = df[targ_cols].values.astype('float32')
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx]), torch.tensor(self.y[idx])

In [18]:
class MultiOPPolymerNet(nn.Module):
    def __init__(self, inp_dim, out_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(inp_dim, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64)
        )
        self.head = nn.Linear(64, out_dim)
        self._initialize_weights()
        
    def forward(self, x):
        x = self.net(x)
        return self.head(x)
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

In [19]:
def train_multiop_model(df, feat_cols, targ_cols, n_epochs=20, save_path = "saved_models/multi_op.pt"):
    df_clean = df[feat_cols + targ_cols].dropna()
    X_scaler = StandardScaler()
    df_clean[feat_cols] = X_scaler.fit_transform(df_clean[feat_cols])
    
    #save input scaler
    with open("saved_models/input_scaler.pkl", 'wb') as f:
        pickle.dump(X_scaler, f)
        
    y_scaler = StandardScaler()
    df_clean[targ_cols] = y_scaler.fit_transform(df_clean[targ_cols])
    
    # save target scaler
    with open("saved_models/output_scaler.pkl", 'wb') as f:
        pickle.dump(y_scaler, f)
        
    train_df, val_df = train_test_split(df_clean, test_size=0.2, random_state=42)
    train_ds = MultiOPDataset(train_df, feat_cols, targ_cols)
    val_ds = MultiOPDataset(val_df, feat_cols, targ_cols)
    train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
    val_dl = DataLoader(train_ds, batch_size=64, shuffle=False)
    
    model = MultiOPPolymerNet(inp_dim=len(feat_cols), out_dim=len(targ_cols)).to(device)
    opt = Adam(model.parameters(), lr=1e-3)
    loss_fn = F.mse_loss
    
    for epoch in range(n_epochs):
        model.train()
        for xb, yb in train_dl:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb)
            loss = loss_fn(preds, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
        
        model.eval()
        val_preds, val_true = [], []
        with torch.no_grad():
            for xb, yb in val_dl:
                xb, yb = xb.to(device), yb.to(device)
                preds = model(xb)
                val_preds.extend(preds.detach().cpu().numpy())
                val_true.extend(yb.detach().cpu().numpy())
        
        maes = {t: mean_absolute_error(val_true[:,i], val_preds[:,i]) for i, t in enumerate(targ_cols)}
        print(f"Epoch {epoch}/{n_epochs}: " + ", ".join([f"{t}: {maes[t]:.4f}" for t in targets]))

    torch.save(model.state_dict(), save_path)
    return model

In [20]:
def predict_multiop_model(test_df, feat_cols, targ_cols, model_path="saved_models/multi_op.pt"):
    with open("saved_models/input_scaler.pkl", "rb") as f:
        X_scaler = pickle.load(f)
    with open("saved_models/output_scaler.pkl", "rb") as f:
        y_scaler = pickle.load(f)

    test_scaled = X_scaler.transform(test_df[feat_cols])
    X_tensor = torch.tensor(test_scaled.astype('float32')).to(device)

    model = MultiOPPolymerNet(inp_dim=len(feat_cols), out_dim=len(targ_cols)).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    with torch.no_grad():
        y_pred = model(X_tensor).cpu().numpy()
    y_pred = y_scaler.inverse_transform(y_pred)
    
    return pd.concat([test_df['id'], pd.DataFrame(y_pred, columns=targets)], axis=1)

In [21]:
maccs_data_test = []

for smile in tqdm(test['SMILES']):
    feats = compute_rdkit_feats(smile)
    if feats is None:
        maccs_data_test.append(None)
    else:
        maccs_data_test.append(feats)
        
maccs_df_test = pd.DataFrame(maccs_data_test)
test_rdkit = pd.concat([test, maccs_df_test], axis=1)

  0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
test_rdkit.head()

,id,SMILES,MACCS_1,MACCS_2,MACCS_3,MACCS_4,MACCS_5,MACCS_6,MACCS_7,MACCS_8,...,MolWt,TPSA,NumValenceElectrons,NumHeavyAtoms,NumRings,NumRotatableBonds,MolLogP,MolMR,NumHAcceptors,NumHDonors
0,1109053969,*Oc1ccc(C=NN=Cc2ccc(Oc3ccc(C(c4ccc(*)cc4)(C(F)...,0,0,0,0,0,0,0,0,...,540.463,43.18,198,39,4,8,7.3603,134.182,4,0
1,1422188626,*Oc1ccc(C(C)(C)c2ccc(Oc3ccc(C(=O)c4cccc(C(=O)c...,0,0,0,0,0,0,0,0,...,510.589,52.60,190,39,5,9,7.2845,151.539,4,0
2,2032016830,*c1cccc(OCCCCCCCCOc2cccc(N3C(=O)c4ccc(-c5cccc6...,0,0,0,0,0,0,0,0,...,586.644,93.22,220,44,6,13,6.1875,164.728,6,0


In [24]:
feat_cols = [c for c in train_rdkit.columns if c.startswith('MACCS_') or c in ['MolWt', 'TPSA', 'NumValenceElectrons', 'NumHeavyAtoms', 'NumRings', 'NumRotatableBonds', 'MolLogP', 'MolMR', 'NumHAcceptors', 'NumHDonors']]
targ_cols = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = train_multiop_model(train_rdkit, feat_cols, targ_cols, n_epochs=30)

ValueError: Found array with 0 sample(s) (shape=(0, 176)) while a minimum of 1 is required by StandardScaler.